In [5]:
import os
import requests
from bs4 import BeautifulSoup as BS
from pprint import pprint

def check_tag(tag, subtag, check):
    try:
        if check in tag[subtag]:
            return True
        return False
    except KeyError:
        return False
    
pos_trans = {'ILB': ['ILB','LB'],
            'OLB': ['OLB','LB'],
            'FS': ['S','DB'],
            'SS': ['S','DB'],
            'OT': ['OT', 'OL', 'T'],
            'OG': ['OG', 'OL','G'],
            'C': ['C', 'OL'],
            'QB': ['QB'],
            'RB': ['RB'],
            'WR': ['WR'],
            'TE': ['TE'],
            'CB': ['CB', 'DB'],
            'DE': ['DE'],
            'DT': ['DT'],
            'FB': ['FB']}

s = '['
for p in pos_trans:
    s += '\'' + p + '\','
s += ']'

print(s)

['DT','QB','OG','C','WR','RB','FB','CB','FS','TE','DE','SS','OLB','ILB','OT',]


In [2]:
CDo = open('COMBINE_DATA.csv')
full = [cd.split(',') for cd in CDo.read().split('\n')]
headers = full[0]
CD = full[1:-1]

base_url = "http://www.pro-football-reference.com/search/search.fcgi?hint=%s+%s&search=%s+%s&pid="
pf_url = "http://www.pro-football-reference.com"
url_dict = {}

final_str = ''
for header in headers:
    final_str += header + ','
final_str += 'G,GS,G3,GS3\n'

for player in range(len(CD)):
    if player % 100 == 0:
        print('\n%s / 3124\n' % str(player))
    name = CD[player][1].replace('\\','')
    CD[player][1] = name
    ns = name.split(' ')
    if len(ns) != 2:
        ns = [ns[0], '%s+%s' % (ns[1],ns[2])]
        
    player_url = base_url % (ns[0], ns[1], ns[0], ns[1])
    response = requests.get(player_url)
    html = response.text.encode('utf-8').__str__()
    n = html.find("<html")
    player_soup = BS(html[n:], "html.parser")
    title_list = player_soup.find_all('title')
    
    player_page = 1
    if 'Search Results' in title_list[0].contents[0]:
        player_page = 0
        
    if not player_page:
        div_list = player_soup.find_all('div')
        for div in div_list:
            if check_tag(div, 'id', 'players'):
                div_list2 = div.find_all('div')
                if div_list2:
                    for div2 in div_list2:
                        try:
                            pos = div2.div.contents[2].split(' ')[2]
                            year = div2.div.contents[2].split(' ')[3][1:5]
                            if pos in pos_trans[CD[player][3]]:
                                if year == CD[player][0]:
                                    player_url = pf_url + div2.div.a['href']
                                    response = requests.get(player_url)
                                    html = response.text.encode('utf-8').__str__()
                                    n = html.find("<html")
                                    player_soup = BS(html[n:], "html.parser")
                                    title_list = player_soup.find_all('title')
                                    break
                        except AttributeError:
                            pass
                    break
    
    resl = [0,0,0,0,0]
    
    if 'Search Results' not in title_list[0].contents[0]:
        table_list = player_soup.find_all('table')
        for table in table_list:
            if check_tag(table, 'class', 'row_summable') and check_tag(table, 'class', 'sortable'):
                th_list = table.thead.find_all('th')
                year = False
                for th in th_list:
                    if check_tag(th, 'aria-label', 'Year'):
                        year = True
                        break
                if year:
                    tr_list = table.tbody.find_all('tr')
                    for tr in tr_list:
                        td_list = tr.find_all('td')
                        if len(tr.th.contents) > 0:
                            if tr.th.contents[0] == '2016':
                                resl[4] += (10/16)
                            else:
                                resl[4] += 1
                        for td in td_list:
                            if check_tag(td, 'data-stat', 'gs'):
                                if len(td.contents) > 0:
                                    gs = int(td.contents[0])
                                    resl[1] += gs
                                    if resl[4] > 3:
                                        resl[3] += gs
                            elif td['data-stat'][0] == 'g':
                                if len(td.contents) > 0:
                                    g = int(td.contents[0])
                                    resl[0] += g
                                    if resl[4] > 3:
                                        resl[2] += g
    
    res0, res1, res2, res3 = 0, 0, 0, 0
    if resl[4] > 0:
        res0 = resl[0]/resl[4]
        res1 = resl[1]/resl[4]
        if resl[4] > 3:
            res2 = resl[2]/(resl[4] - 3)
            res3 = resl[3]/(resl[4] - 3)
    print(name, res0, res1, res2, res3, resl[4])
    CD[player].append(str(res0))
    CD[player].append(str(res1))
    CD[player].append(str(res2))
    CD[player].append(str(res3))
    for p in CD[player]:
        final_str += p + ','
    final_str += '\n'

print('DONE')


0 / 3124

Oday Aboushi 8.0 5.666666666666667 0 0 3
Robert Alford 13.5 10.5 13.0 13.0 4
C.J. Anderson 10.5 4.75 7.0 7.0 4
Ezekiel Ansah 14.0 13.5 10.0 10.0 4
Marc Anthony 0 0 0 0 0
Terron Armstead 10.0 9.0 7.0 7.0 4
Tavon Austin 14.0 9.25 12.0 12.0 4
Stedman Bailey 12.666666666666666 2.6666666666666665 0 0 3
David Bakhtiari 14.75 14.75 13.0 13.0 4
Montee Ball 10.5 1.5 0 0 2
Johnthan Banks 16.275862068965516 10.206896551724139 22.4 0.0 3.625
Chris Barker 0 0 0 0 0
Kenjon Barner 10.333333333333334 0.0 0 0 3
T.J. Barnes 0 0 0 0 0


KeyboardInterrupt: 

In [72]:
o = open('OUTPUT.csv', 'w')
o.write(final_str)
o.close()

/players/P/PeteAd00.htm
/players/P/PeteAd01.htm
